In [1]:
import pandas as pd
import numpy as np
import botocore.exceptions as botoexceptions
import os
import sys
import json
import logging.config
import yaml
import argparse
import glob

# Logging
logging.basicConfig(format='%(asctime)s %(levelname)-8s %(message)s', level=logging.INFO, datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)


In [2]:
trips = pd.read_csv('/Users/Brian/git/423_avc/2021-msia423-lewis-brian-project/data/from_s3/trips.csv')
stations = pd.read_csv('/Users/Brian/git/423_avc/2021-msia423-lewis-brian-project/data/from_s3/stations.csv')

In [3]:
bikes = pd.merge(trips, stations, 
                 how='left', 
                 left_on = ['station_id'], 
                 right_on = ['station_id']).\
                 sort_values(['station_id', 'date', 'hour']).\
                 drop(['num_bikes_available', 'last_reported'], axis=1)

bikes['name'] = bikes.groupby('station_id')['name'].transform(lambda x: x.ffill().bfill())
bikes['latitude'] = bikes.groupby('station_id')['latitude'].transform(lambda x: x.ffill().bfill())
bikes['longitude'] = bikes.groupby('station_id')['longitude'].transform(lambda x: x.ffill().bfill())
bikes['capacity'] = bikes.groupby('station_id')['capacity'].transform(lambda x: x.ffill().bfill())
bikes['net_flows'] = bikes['outflows'] - bikes['inflows']

# Come back to https://feeds.citibikenyc.com/stations/stations.json if I have time.

In [4]:
test_df = bikes
# test_df

test_df1 = []

In [5]:
# test_df1.groupby(['station_id', 'date'])['hour'].transform('min')==test_df1['hour']

test_df1 = test_df
test_df1['stock'] = np.where(test_df1.groupby(['station_id', 'date'])['hour'].transform('min') == test_df1['hour'],
                             round(test_df1.capacity * 0.65),
                             test_df1.net_flows.shift()
                            )

test_df1['stock'] = test_df1.groupby(['station_id', 'date'])['stock'].cumsum()
test_df1['date'] = pd.to_datetime(test_df1.date)
test_df1['date'] += pd.to_timedelta(test_df1.hour, unit = 'h')

test_df2 = test_df1.drop(['capacity', 'inflows', 'hour', 'outflows', 'net_flows'], axis=1)

test_df2

,station_id,date,name,latitude,longitude,stock
768606,72,2020-03-01 06:00:00,W 52 St & 11 Ave,40.767272,-73.993929,36.0
768607,72,2020-03-01 09:00:00,W 52 St & 11 Ave,40.767272,-73.993929,38.0
768608,72,2020-03-01 10:00:00,W 52 St & 11 Ave,40.767272,-73.993929,35.0
768609,72,2020-03-01 11:00:00,W 52 St & 11 Ave,40.767272,-73.993929,37.0
768610,72,2020-03-01 12:00:00,W 52 St & 11 Ave,40.767272,-73.993929,37.0
...,...,...,...,...,...,...
3834233,4435,2021-03-30 20:00:00,4 Ave & 17 St,40.665507,-73.993037,13.0
3834234,4435,2021-03-31 11:00:00,4 Ave & 17 St,40.665507,-73.993037,15.0
3834235,4435,2021-03-31 19:00:00,4 Ave & 17 St,40.665507,-73.993037,16.0
3834236,4436,2021-03-31 18:00:00,Park Cir & East Dr,40.651566,-73.972212,19.0


In [212]:
len(test_df2.name.unique())

1326

In [7]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install chart_studio
# !{sys.executable} -m pip install statsmodels

from chart_studio.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose

d1 = test_df2.query('station_id == 72')

d2 = d1[['date', 'stock']]

d2.set_index('date')

d2
# result = seasonal_decompose(d2, model='additive', period = 24)
# fig = result.plot()
# plot_mpl(fig)

,date,stock
768606,2020-03-01 06:00:00,36.0
768607,2020-03-01 09:00:00,38.0
768608,2020-03-01 10:00:00,35.0
768609,2020-03-01 11:00:00,37.0
768610,2020-03-01 12:00:00,37.0
...,...,...
3517317,2021-03-31 16:00:00,53.0
3517318,2021-03-31 17:00:00,53.0
3517319,2021-03-31 19:00:00,56.0
3517320,2021-03-31 20:00:00,52.0


In [121]:
bikes_filtered = bikes[bikes['name'].notnull()].drop(['capacity', 'last_reported'], axis=1)

In [135]:
# bikes_filtered.query("num_bikes_available > 0").head(20)

In [123]:
bike_stock = bikes_filtered.loc[bikes_filtered.query("num_bikes_available > 0").\
                   groupby('station_id').hour.idxmin()]

In [134]:
# bike_stock.head(10)

In [133]:
bikes_filtered2 = bikes_filtered.drop(['num_bikes_available'], axis = 1)
# bikes_filtered2.head(10)

In [147]:
bikes_df = pd.merge(bikes_filtered2, bike_stock, how='left')
bikes_df['net_flows'] = bikes_df['outflows'] - bikes_df['inflows']

In [154]:
# # chop off data to be at the station level only those observations with bike stock or later in time
# bikes_g1 = bikes_df.groupby('station_id')

bikes_df1 = bikes_df
bikes_df1['cumsum'] = bikes_df1.groupby('station_id')['net_flows'].cumsum()

bikes_df1.query("date == '2020-04-14'").head(35)



,station_id,date,hour,inflows,outflows,name,latitude,longitude,num_bikes_available,net_flows,stock,cumsum
37333,72.0,2020-04-14,7,1,2,W 52 St & 11 Ave,40.767272,-73.993929,48.0,1,NaN,2542
37334,72.0,2020-04-14,8,1,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,49.0,2542
37335,72.0,2020-04-14,9,2,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,-1,NaN,2541
37336,72.0,2020-04-14,11,2,4,W 52 St & 11 Ave,40.767272,-73.993929,NaN,2,NaN,2543
37337,72.0,2020-04-14,12,1,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,2,NaN,2545
37338,72.0,2020-04-14,14,2,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,1,NaN,2546
37339,72.0,2020-04-14,15,3,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,NaN,2546
37340,72.0,2020-04-14,16,3,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,NaN,2546
37341,72.0,2020-04-14,17,3,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,-2,NaN,2544
37342,72.0,2020-04-14,18,1,8,W 52 St & 11 Ave,40.767272,-73.993929,NaN,7,NaN,2551


In [153]:
# bikes_df1 = bikes_df
# bikes_df1['stock'] = np.where(bikes_df1.groupby('station_id')['num_bikes_available'].shift() >= 0, 
#                               bikes_df1.groupby('station_id')['num_bikes_available'].shift(),
#                               np.nan) + \
#                               bikes_df1.groupby('station_id')['net_flows'].shift()
    
# bikes_df1.query("date == '2020-04-14'").head(35)


,station_id,date,hour,inflows,outflows,name,latitude,longitude,num_bikes_available,net_flows,stock
37333,72.0,2020-04-14,7,1,2,W 52 St & 11 Ave,40.767272,-73.993929,48.0,1,NaN
37334,72.0,2020-04-14,8,1,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,49.0
37335,72.0,2020-04-14,9,2,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,-1,NaN
37336,72.0,2020-04-14,11,2,4,W 52 St & 11 Ave,40.767272,-73.993929,NaN,2,NaN
37337,72.0,2020-04-14,12,1,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,2,NaN
37338,72.0,2020-04-14,14,2,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,1,NaN
37339,72.0,2020-04-14,15,3,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,NaN
37340,72.0,2020-04-14,16,3,3,W 52 St & 11 Ave,40.767272,-73.993929,NaN,0,NaN
37341,72.0,2020-04-14,17,3,1,W 52 St & 11 Ave,40.767272,-73.993929,NaN,-2,NaN
37342,72.0,2020-04-14,18,1,8,W 52 St & 11 Ave,40.767272,-73.993929,NaN,7,NaN


In [ ]:

def process_bike_data(trips_df, stations_df):
    """
    Filters raw data to columns of interest and joins trips and stations datasets
    Args:
        trips_df (pandas DataFrame): DataFrame consisting of the trips data acquired from Citi Bike
        stations_df (pandas DataFrame): DataFrame consisting of the stations data acquired from Citi Bike
    Returns:
        df (pandas DataFrame): Reduced columns version of the input DataFrame
    """
    try:
        # Joined data
        trips_df = trips_df

        # ARIMA modeling

    except AttributeError:
        logger.error(
            "Your input to the function 'process_bike_data' was not a DataFrame and thus the function could not run")
        sys.exit(1)
    except Exception as e:
        logger.error("Unexpected error in the function 'process_bike_data': {}:{}".format(type(e).__name__, e))
        sys.exit(1)

    return bike_df